# Fed-BioMed SecAgg Joye-Libert

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"]
        
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


In [3]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

In [4]:
from fedbiomed.researcher.experiment_jl import ExperimentJL
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 10

exp = ExperimentJL(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-09-09 17:23:46,938 fedbiomed INFO - Component environment:
2022-09-09 17:23:46,939 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-09-09 17:23:47,012 fedbiomed INFO - Messaging researcher_f07a5365-9039-453c-94c8-41b2ad3a2c8c successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f436596b6a0>
2022-09-09 17:23:47,049 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-09-09 17:23:57,061 fedbiomed INFO - Node selected for training -> node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49
2022-09-09 17:23:57,063 fedbiomed INFO - Node selected for training -> node_be002608-84c9-4f81-ade8-24f31fca87dd
2022-09-09 17:23:57,065 fedbiomed INFO - Checking data quality of federated datasets...
2022-09-09 17:23:57,081 fedbiomed DEBUG - Model file has been saved: /workspaces/PycharmProjects/fedbiomed/var/experiments/Experiment_0034/my_model_55a2c903-1210-416d-8efa-509aae14efd1.py
2022-09-09 17:23:57,100 fedbiom

In [5]:
exp.run(rounds=rounds, increase=True)

2022-09-09 17:23:57,264 fedbiomed INFO - Sampled nodes in round 0 ['node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49', 'node_be002608-84c9-4f81-ade8-24f31fca87dd']
2022-09-09 17:23:57,265 fedbiomed INFO - Sending request 
					 To: node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_f07a5365-9039-453c-94c8-41b2ad3a2c8c', 'job_id': '5419cb92-32e4-440e-9917-d84859e01023', 'current_round': 0, 'training_args': scheme:
{'lr': {'rules': [<class 'float'>, <function TrainingArgs._lr_hook at 0x7f4365aa1820>], 'required': False}, 'batch_size': {'rules': [<class 'int'>], 'required': False}, 'epochs': {'rules': [<class 'int'>], 'required': False}, 'dry_run': {'rules': [<class 'bool'>], 'required': False}, 'batch_maxnum': {'rules': [<class 'int'>], 'required': False}, 'test_ratio': {'rules': [<class 'float'>, <function TrainingArgs._test_ratio_hook at 0x7f4365aa1700>], 'required': False, 'default': 0.0}, 'test_on_local_updates': {'

2022-09-09 17:24:02,610 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49 
					 Epoch: 1 | Completed: 3840/60000 (6%) 
 					 Loss: 0.339639 
					 ---------
2022-09-09 17:24:03,228 fedbiomed INFO - TRAINING 
					 NODE_ID: node_be002608-84c9-4f81-ade8-24f31fca87dd 
					 Epoch: 1 | Completed: 4320/60000 (7%) 
 					 Loss: 0.512393 
					 ---------
2022-09-09 17:24:03,231 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49 
					 Epoch: 1 | Completed: 4320/60000 (7%) 
 					 Loss: 0.499359 
					 ---------
2022-09-09 17:24:03,815 fedbiomed INFO - INFO
					 NODE node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49
					 MESSAGE: Reached 100 batches for this epoch, ignore remaining data
-----------------------------------------------------------------
2022-09-09 17:24:03,823 fedbiomed INFO - INFO
					 NODE node_be002608-84c9-4f81-ade8-24f31fca87dd
					 MESSAGE: Reached 100 batches for this epoch, ignore remaining data
-----------

2022-09-09 17:24:13,886 fedbiomed DEBUG - researcher_f07a5365-9039-453c-94c8-41b2ad3a2c8c
2022-09-09 17:24:13,922 fedbiomed INFO - WARNING
					 NODE node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49
					 MESSAGE: There is no validation activated for the round. Please set flag for `test_on_global_updates`, `test_on_local_updates`, or both. Splitting dataset for validation will be ignored
-----------------------------------------------------------------
2022-09-09 17:24:13,946 fedbiomed INFO - INFO
					 NODE node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49
					 MESSAGE: training with arguments {'history_monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x7f74781fcb50>, 'node_args': {'gpu': False, 'gpu_num': None, 'gpu_only': False}, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}
-----------------------------------------------------------------
2022-09-09 17:24:13,987 fedbiomed INFO - WARNING
					 NODE node_be002608-84c9-4f81-ade8-24f31fca87dd
					 MESSAGE: There

2022-09-09 17:24:23,551 fedbiomed INFO - INFO
					 NODE node_be002608-84c9-4f81-ade8-24f31fca87dd
					 MESSAGE: results uploaded successfully 
-----------------------------------------------------------------
2022-09-09 17:24:28,903 fedbiomed INFO - Downloading model params after training on node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49 - from http://localhost:8844/media/uploads/2022/09/09/node_params_6a6122d8-83d8-43dc-8201-aa85aeeb7221.pkl
2022-09-09 17:24:28,946 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_b972b093-5a56-4438-8950-728d74d1a321.pkl successful, with status code 200
2022-09-09 17:24:28,988 fedbiomed INFO - Model params downloaded
2022-09-09 17:24:28,989 fedbiomed INFO - Downloading model params after training on node_be002608-84c9-4f81-ade8-24f31fca87dd - from http://localhost:8844/media/uploads/2022/09/09/node_params_317789c9-dbc1-4bcf-a66b-ffffa75c13fb.pkl
2022-09-09 17:24:29,000 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_53b6ee09

2022-09-09 17:24:33,486 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49 
					 Epoch: 1 | Completed: 1920/60000 (3%) 
 					 Loss: 0.267688 
					 ---------
2022-09-09 17:24:33,939 fedbiomed INFO - TRAINING 
					 NODE_ID: node_be002608-84c9-4f81-ade8-24f31fca87dd 
					 Epoch: 1 | Completed: 1920/60000 (3%) 
 					 Loss: 0.196055 
					 ---------
2022-09-09 17:24:34,140 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49 
					 Epoch: 1 | Completed: 2400/60000 (4%) 
 					 Loss: 0.172968 
					 ---------
2022-09-09 17:24:34,561 fedbiomed INFO - TRAINING 
					 NODE_ID: node_be002608-84c9-4f81-ade8-24f31fca87dd 
					 Epoch: 1 | Completed: 2400/60000 (4%) 
 					 Loss: 0.111594 
					 ---------
2022-09-09 17:24:34,781 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49 
					 Epoch: 1 | Completed: 2880/60000 (5%) 
 					 Loss: 0.199818 
					 ---------
2022-09-09 17:24:35,169 fedbiomed I

...

2022-09-09 17:26:07,697 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49 
					 Epoch: 1 | Completed: 1920/60000 (3%) 
 					 Loss: 0.069566 
					 ---------
2022-09-09 17:26:07,888 fedbiomed INFO - TRAINING 
					 NODE_ID: node_be002608-84c9-4f81-ade8-24f31fca87dd 
					 Epoch: 1 | Completed: 1920/60000 (3%) 
 					 Loss: 0.136873 
					 ---------
2022-09-09 17:26:08,553 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49 
					 Epoch: 1 | Completed: 2400/60000 (4%) 
 					 Loss: 0.109871 
					 ---------
2022-09-09 17:26:08,597 fedbiomed INFO - TRAINING 
					 NODE_ID: node_be002608-84c9-4f81-ade8-24f31fca87dd 
					 Epoch: 1 | Completed: 2400/60000 (4%) 
 					 Loss: 0.202878 
					 ---------
2022-09-09 17:26:09,319 fedbiomed INFO - TRAINING 
					 NODE_ID: node_be002608-84c9-4f81-ade8-24f31fca87dd 
					 Epoch: 1 | Completed: 2880/60000 (5%) 
 					 Loss: 0.094113 
					 ---------
2022-09-09 17:26:09,324 fedbiomed I

2022-09-09 17:26:26,485 fedbiomed DEBUG - researcher_f07a5365-9039-453c-94c8-41b2ad3a2c8c
2022-09-09 17:26:26,487 fedbiomed INFO - Sending request 
					 To: node_be002608-84c9-4f81-ade8-24f31fca87dd 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_f07a5365-9039-453c-94c8-41b2ad3a2c8c', 'job_id': '5419cb92-32e4-440e-9917-d84859e01023', 'current_round': 8, 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100, 'test_ratio': 0.0, 'test_on_local_updates': False, 'test_on_global_updates': False, 'test_metric': None, 'test_metric_args': {}}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/09/09/my_model_55a2c903-1210-416d-8efa-509aae14efd1.py', 'params_url': 'http://localhost:8844/media/uploads/2022/09/09/aggregated_params_ddf673a5-2516-4001-a817-7c99dbc9fd08.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_be002608-84c9-4f81-ade8-24f3

2022-09-09 17:26:35,802 fedbiomed INFO - INFO
					 NODE node_be002608-84c9-4f81-ade8-24f31fca87dd
					 MESSAGE: Quantization of model parameters done
-----------------------------------------------------------------
2022-09-09 17:26:35,807 fedbiomed INFO - INFO
					 NODE node_be002608-84c9-4f81-ade8-24f31fca87dd
					 MESSAGE: Protecting model parameters with key <UserKey 0xa>
-----------------------------------------------------------------
2022-09-09 17:26:36,353 fedbiomed INFO - INFO
					 NODE node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49
					 MESSAGE: Protecting model parameters done
-----------------------------------------------------------------
2022-09-09 17:26:36,525 fedbiomed INFO - INFO
					 NODE node_be002608-84c9-4f81-ade8-24f31fca87dd
					 MESSAGE: Protecting model parameters done
-----------------------------------------------------------------
2022-09-09 17:26:36,630 fedbiomed INFO - INFO
					 NODE node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49
					 MESSAGE: results upl

2022-09-09 17:26:49,101 fedbiomed INFO - TRAINING 
					 NODE_ID: node_be002608-84c9-4f81-ade8-24f31fca87dd 
					 Epoch: 1 | Completed: 480/60000 (1%) 
 					 Loss: 0.038713 
					 ---------
2022-09-09 17:26:49,657 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49 
					 Epoch: 1 | Completed: 960/60000 (2%) 
 					 Loss: 0.067271 
					 ---------
2022-09-09 17:26:49,896 fedbiomed INFO - TRAINING 
					 NODE_ID: node_be002608-84c9-4f81-ade8-24f31fca87dd 
					 Epoch: 1 | Completed: 960/60000 (2%) 
 					 Loss: 0.076669 
					 ---------
2022-09-09 17:26:50,490 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fe9c6fd7-b931-4ef3-baf8-3a219d264e49 
					 Epoch: 1 | Completed: 1440/60000 (2%) 
 					 Loss: 0.006504 
					 ---------
2022-09-09 17:26:50,774 fedbiomed INFO - TRAINING 
					 NODE_ID: node_be002608-84c9-4f81-ade8-24f31fca87dd 
					 Epoch: 1 | Completed: 1440/60000 (2%) 
 					 Loss: 0.037201 
					 ---------
2022-09-09 17:26:51,376 fedbiomed INFO

10